# Accessing and Querying the Survey Class

The survey HDF5 file is a master look up table for each observation made by VIRUS for HETDEX. It can be accessed directly through the HDF5 survey file, or through the Survey class API. The Survey class includes information for each three dither set HETDEX observation in HDR1. It includes both science and calibration shots. Any quantity related to an individual dither is provided in an array of length three. Fundamental properties of a shot and quality assessment values for seeing (fwhm), astrometry, flux limits and transparency (response_4540) are tabulated to allow for querying of desired shots. 

We first introduce the basic API to query the Survey Class by sky coordinates to get a list of shots where fibers are located within that region. Later on in this notebook we show some basic querying of the Survey class and demonstrate a few examples.

### Initiate the Survey class :

In [1]:
%matplotlib inline
import tables as tb
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord

from hetdex_api.survey import Survey, FiberIndex

In [2]:
S = Survey('pdr1')

In [3]:
# convert to an astropy table format. Can remove any flagged shots if desired by setting return_good=True.
survey_table = S.return_astropy_table(return_good=False) 

In [4]:
survey_table

shotid,field,objid,date,obsid,ra,dec,pa,n_ifu,fwhm_virus,fwhm_virus_err,response_4540,ambtemp,datevobs,dewpoint,exptime,humidity,mjd,nstars_fit_fwhm,obsind,pressure,structaz,time,trajcdec,trajcpa,trajcra,shot_flag
int64,bytes12,bytes18,int32,int32,float64,float64,float64,int32,float32,float32,float32,float32,bytes12,float32,float32,float32,float32,int32,int32,float32,float32,bytes7,float32,float32,float32,bool
20170920008,dex-fall,DEX00068_0_000_E,20170920,8,24.588702,0.000469,154.144974,21,1.6656576,0.40102777,0.1212,20.422777,20170920v008,10.840555,367.09952,52.572,58016.305,8,1,802.2015,147.76714,0717562,0.0,152.69672,24.5895,True
20170920009,dex-fall,DEX00094_0_000_E,20170920,9,31.608569,0.000689,154.184946,21,1.9848523,0.18202426,0.1006,20.372223,20170920v009,10.787222,367.59937,52.337,58016.324,4,1,802.2015,147.78941,0745268,0.0,152.71495,31.6095,True
20170921018,dex-fall,DEX00001_0,20170921,18,6.498122,0.000526,154.177847,18,1.7578322,0.5647613,0.1374,20.896667,20170921v018,9.74,367.166,46.673,58017.25,7,1,803.082,147.73189,0559186,0.0,152.66785,6.4995,True
20170921020,dex-fall,DEX00046_0_000_E,20170921,20,18.649292,0.001456,154.126565,18,1.6957773,0.27283788,0.113,20.472221,20170921v020,10.080556,367.06583,49.737,58017.285,12,1,803.082,147.75246,0647548,0.0,152.6847,18.6495,True
20170921021,dex-fall,DEX00070_0_000_E,20170921,21,25.127779,0.000856,154.16784,18,1.6449038,0.18297766,0.118,20.043333,20170921v021,10.283889,366.5493,51.741,58017.3,4,1,803.0481,147.76953,0712281,0.0,152.69867,25.1295,True
20170921022,dex-fall,DEX00070_0_000_E,20170921,22,31.338945,0.000353,154.178197,18,1.9767525,0.5830754,0.1277,19.638334,20170921v022,10.735556,367.19894,54.857,58017.316,7,1,803.0481,147.78941,0736586,0.0,152.71495,31.3395,True
20170921025,dex-fall,DEX00017_1_000_W,20170921,25,10.820815,0.000884,208.837196,18,1.7373962,0.14883323,0.0892,19.426111,20170921v025,10.836667,366.96646,56.025,58017.36,4,1,802.91266,212.2687,0838279,0.0,207.33266,10.8195,True
20170921026,dex-fall,DEX00041_1_000_W,20170921,26,17.300609,0.001184,208.919681,18,1.7994181,0.013720359,0.116,18.890556,20170921v026,10.827222,367.54956,58.009,58017.38,5,1,802.7095,212.24982,0905028,0.0,207.31718,17.2995,True
20170921029,dex-fall,DEX00095_1_000_W,20170921,29,31.878935,0.000924,208.749334,18,1.8989948,0.898437,0.1213,18.762777,20170921v029,10.489445,367.14957,57.425,58017.42,6,1,802.1676,212.20863,1006376,0.0,207.28346,31.8795,True


It also converts RA and DEC into astropy coordinate objects.

In [5]:
S.coords

<SkyCoord (ICRS): (ra, dec) in deg
    [( 24.588702, 4.6900000e-04), ( 31.608569, 6.8900000e-04),
     (  6.498122, 5.2600000e-04), ..., (240.071344, 5.2579344e+01),
     (240.801501, 5.2778936e+01), (244.692454, 5.0994920e+01)]>

Shots are labeled based on their observation date and observation ID:

In [6]:
S.date

array([20170920, 20170920, 20170921, ..., 20210826, 20210828, 20210830],
      dtype=int32)

In [7]:
S.obsid

array([ 8,  9, 18, ...,  8,  7, 13], dtype=int32)

A specific shot can be linked either by an integer combination of these two values, called `shotid`, or the `DATEvOBS` form:

In [8]:
S.datevobs

array(['20170920v008', '20170920v009', '20170921v018', ...,
       '20210826v008', '20210828v007', '20210830v013'], dtype='<U12')

In [9]:
S.shotid

array([20170920008, 20170920009, 20170921018, ..., 20210826008,
       20210828007, 20210830013])

## Searching for shots near a sky coordinate

Most users searching for observations will want to get a list of observations/shots at a specific sky position. Each shot has a single associated sky coordinate. We can retrieve a list of shots that may contain fibers in a specific region. However, because of the varying and non-contiguous footprint of VIRUS, this is not an immediate guarantee a region has been observed. You must further query fibers in each shot to determine this. See Notebook 02 to learn how to query with the Fibers Class API. 

### get_shotlist()
This function which operates on the Survey class object. An astropy coordinates object is required as input.

In [10]:
coords = SkyCoord(11.628530 * u.deg, 0.081790 * u.deg, frame='icrs')

We can either search a rectangular region, width and height in degrees:

In [11]:
shotlist = S.get_shotlist(coords, width=0.5, height=0.2)

In [12]:
shotlist

array([20171119003, 20180106004, 20181003009, 20181005017, 20181107014,
       20191222012])

or we can search a circular aperture with a radius given in degress, or in an astropy quantity object:

In [13]:
shotlist = S.get_shotlist(coords, radius=10*u.arcmin)

In [14]:
shotlist

array([20171119003, 20180106004, 20181003009, 20181107014, 20191222012])

From here a user can take their shotlist and query the position on the Fibers class and see if their region of interest is observed in the survey.

## Basic querying for the Survey Class

The Survey HDF5 itself contains information about the quality of each observation. A user can query different parameters and plot up general shot quantities. For example, we can compare the distribution of throughput values (technically the response at 4540 AA) between the HETDEX spring and HETDEX fall fields:

### Example: Plotting up the canonical throughput value at 4540 AA

In [15]:
idx_spring = np.where( (S.field == 'dex-spring') * np.isfinite(S.response_4540) )

In [16]:
idx_fall = np.where( (S.field == 'dex-fall') * np.isfinite(S.response_4540))

In [17]:
plt.figure(figsize=(8,6))
plt.hist(S.response_4540[idx_spring], label = 'DEX-Spring', bins=20)
plt.hist(S.response_4540[idx_fall], label = 'DEX-Fall', bins=20)
plt.xlabel('Throughput at 4540 AA')
plt.ylabel('N OBS')
plt.legend()

# Accessing the Master Fiber Index Table:

Each fiber in the HETDEX survey is assigned a coordinate and healpix value. You can efficiently query the fiber database using the FiberIndex class.

In [18]:
FibIndex = FiberIndex()

In [19]:
coords = SkyCoord(11.628530 * u.deg, 0.081790 * u.deg, frame='icrs')

In [20]:
coords.ra.deg

11.62853

You can read in a circular region of the Fiber Index table using the query_region() function:

In [21]:
fiber_table_region = FibIndex.query_region(coords, radius=3.*u.arcsec, shotid=None)

In [22]:
fiber_table_region

multiframe,ra,dec,fiber_id,healpix,amp,date,datevobs,expnum,fibidx,fibnum,fpx,fpy,ifuid,ifuslot,ifux,ifuy,shotid,specid,flag,amp_flag,meteor_flag,gal_flag,shot_flag,throughput_flag
bytes20,float32,float32,bytes38,int64,bytes2,int64,bytes12,int32,int32,int32,float32,float32,bytes3,bytes3,float32,float32,int64,bytes3,bool,bool,bool,bool,bool,bool
multi_308_047_024_LU,11.629047,0.0819793,20181003009_1_multi_308_047_024_LU_006,6433083530,LU,20181003,20181003v009,1,5,6,-251.53,-162.94,024,047,-1.27,-13.22,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LU,11.628714,0.0822275,20181003009_3_multi_308_047_024_LU_007,6433083530,LU,20181003,20181003v009,3,6,7,-252.855,-162.24,024,047,-3.81,-13.22,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LL,11.628361,0.08245,20181003009_2_multi_308_047_024_LL_100,6433083530,LL,20181003,20181003v009,2,99,100,-254.125,-161.44,024,047,-5.08,-11.02,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LU,11.629065,0.0823952,20181003009_2_multi_308_047_024_LU_007,6433083530,LU,20181003,20181003v009,2,6,7,-252.855,-163.64,024,047,-3.81,-13.22,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LL,11.627942,0.0814511,20181003009_1_multi_308_047_024_LL_079,6433214601,LL,20181003,20181003v009,1,78,79,-251.53,-158.53,024,047,-1.27,-8.81,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LL,11.628648,0.0813975,20181003009_1_multi_308_047_024_LL_098,6433214601,LL,20181003,20181003v009,1,97,98,-250.26,-160.74,024,047,0.0,-11.02,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LL,11.628344,0.0820341,20181003009_1_multi_308_047_024_LL_099,6433214601,LL,20181003,20181003v009,1,98,99,-252.8,-160.74,024,047,-2.54,-11.02,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LL,11.628314,0.0816458,20181003009_3_multi_308_047_024_LL_099,6433214601,LL,20181003,20181003v009,3,98,99,-251.585,-160.04,024,047,-2.54,-11.02,20181003009,308,True,True,True,True,True,True
multi_308_047_024_LL,11.62801,0.0822824,20181003009_3_multi_308_047_024_LL_100,6433214601,LL,20181003,20181003v009,3,99,100,-254.125,-160.04,024,047,-5.08,-11.02,20181003009,308,True,True,True,True,True,True


Note: you have the option to request a specific shotid (default is None) or whether to return an astropy table format or not. Default is to return an astropy table.

In [23]:
healpix_table = FibIndex.get_fib_from_hp(6433214603)

In [24]:
healpix_table

multiframe,ra,dec,fiber_id,healpix,amp,date,datevobs,expnum,fibidx,fibnum,fpx,fpy,ifuid,ifuslot,ifux,ifuy,shotid,specid
bytes20,float32,float32,bytes38,int64,bytes2,int64,bytes12,int32,int32,int32,float32,float32,bytes3,bytes3,float32,float32,int64,bytes3
multi_308_047_024_LU,11.632875,0.0810685,20181003009_1_multi_308_047_024_LU_100,6433214603,LU,20181003,20181003v009,1,99,100,-242.63,-173.96,024,047,7.63,-24.24,20181003009,308
multi_308_047_024_LU,11.63257,0.0817076,20181003009_1_multi_308_047_024_LU_101,6433214603,LU,20181003,20181003v009,1,100,101,-245.18,-173.96,024,047,5.08,-24.24,20181003009,308
multi_308_047_024_LU,11.63254,0.0813193,20181003009_3_multi_308_047_024_LU_101,6433214603,LU,20181003,20181003v009,3,100,101,-243.965,-173.26,024,047,5.08,-24.24,20181003009,308
multi_308_047_024_LU,11.632236,0.0819559,20181003009_3_multi_308_047_024_LU_102,6433214603,LU,20181003,20181003v009,3,101,102,-246.505,-173.26,024,047,2.54,-24.24,20181003009,308
multi_308_047_024_LU,11.632185,0.0815406,20181003009_2_multi_308_047_024_LU_082,6433214603,LU,20181003,20181003v009,2,81,82,-245.235,-172.45,024,047,3.81,-22.03,20181003009,308
multi_308_047_024_LU,11.633196,0.0808479,20181003009_2_multi_308_047_024_LU_100,6433214603,LU,20181003,20181003v009,2,99,100,-241.415,-174.66,024,047,7.63,-24.24,20181003009,308
multi_308_047_024_LU,11.632891,0.081487,20181003009_2_multi_308_047_024_LU_101,6433214603,LU,20181003,20181003v009,2,100,101,-243.965,-174.66,024,047,5.08,-24.24,20181003009,308
multi_308_047_024_LU,11.6325865,0.0821236,20181003009_2_multi_308_047_024_LU_102,6433214603,LU,20181003,20181003v009,2,101,102,-246.505,-174.66,024,047,2.54,-24.24,20181003009,308


## Access the survey H5 file

The hdfile attribute contains the link to the survey HDF5 file. Calling it will show you all the information in the HDF5 file. You may interact with this directly rather than using the Survey class initiation. 


In [25]:
S.hdfile

File(filename=/scratch/projects/hetdex/pdr1/survey/survey_pdr1.h5, title='PDR1 Survey File', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'PDR1 Survey File'
/Survey (Table(3502,)) ''
  description := {
  "shotid": Int64Col(shape=(), dflt=0, pos=0),
  "field": StringCol(itemsize=12, shape=(), dflt=b'', pos=1),
  "objid": StringCol(itemsize=18, shape=(), dflt=b'', pos=2),
  "date": Int32Col(shape=(), dflt=0, pos=3),
  "obsid": Int32Col(shape=(), dflt=0, pos=4),
  "ra": Float64Col(shape=(), dflt=0.0, pos=5),
  "dec": Float64Col(shape=(), dflt=0.0, pos=6),
  "pa": Float64Col(shape=(), dflt=0.0, pos=7),
  "n_ifu": Int32Col(shape=(), dflt=0, pos=8),
  "fwhm_virus": Float32Col(shape=(), dflt=0.0, pos=9),
  "fwhm_virus_err": Float32Col(shape=(), dflt=0.0, pos=10),
  "response_4540": Float32Col(shape=(), dflt=0.0, pos=11),
  "ambtemp": Float32Col(shape=(), dflt=0.0, pos=12),
  "datevobs": Str

In [26]:
## Close the Survey H5 file after use

It is good practice to close any h5 files that are open when you no longer need to access new data from the h5 file

In [27]:
S.close()